In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_rows', 100)

from scipy.stats import rankdata

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

In [3]:
def create_validation_features ():
    df_valid_pair = pd.read_csv("../processed/validation.csv")
    df_valid_fe = pd.read_csv("../processed/valid_text_detoxify_fe.csv")

    df_valid_fe_2 = pd.read_csv("../processed/valid_text_jc_tfidf_fe.csv")
    df_valid_fe_2 = df_valid_fe_2[["jc_tfidf_toxic","jc_tfidf_severe_toxic","jc_tfidf_obscene","jc_tfidf_threat","jc_tfidf_insult","jc_tfidf_identity_hate"]]
    
    df_valid_fe_3 = pd.read_csv("../processed/valid_text_juc_tfidf_fe.csv")
    df_valid_fe_3 = df_valid_fe_3[['juc_tfidf_toxicity', 'juc_tfidf_severe_toxicity','juc_tfidf_obscene', 'juc_tfidf_sexual_explicit','juc_tfidf_identity_attack', 'juc_tfidf_insult', 'juc_tfidf_threat']]
    
    df_valid_fe = pd.concat ( [df_valid_fe,df_valid_fe_2, df_valid_fe_3], axis=1)    
    
    return df_valid_pair, df_valid_fe

    
def create_rud_features ():
    df_valid_pair = pd.read_csv("../processed/rud_pair.csv")
    df_valid_fe = pd.read_csv("../processed/rud_text_detoxify_fe.csv")

    df_valid_fe_2 = pd.read_csv("../processed/rud_text_jc_tfidf_fe.csv")
    df_valid_fe_2 = df_valid_fe_2[["jc_tfidf_toxic","jc_tfidf_severe_toxic","jc_tfidf_obscene","jc_tfidf_threat","jc_tfidf_insult","jc_tfidf_identity_hate"]]

    df_valid_fe_3 = pd.read_csv("../processed/rud_text_juc_tfidf_fe.csv")
    df_valid_fe_3 = df_valid_fe_3[['juc_tfidf_toxicity', 'juc_tfidf_severe_toxicity','juc_tfidf_obscene', 'juc_tfidf_sexual_explicit','juc_tfidf_identity_attack', 'juc_tfidf_insult', 'juc_tfidf_threat']]
    
    df_valid_fe = pd.concat ( [df_valid_fe,df_valid_fe_2, df_valid_fe_3], axis=1)    
    
    return df_valid_pair, df_valid_fe


def validate(df_valid_pair,df_valid_text, pred):
    

    ### rud
    if "comment_id" in df_valid_text.columns:
        m = { a:b for a, b in zip(df_valid_text["comment_id"].values, pred)  }
        p_less = df_valid_pair["less_toxic_id"].map(lambda x: m[x] )
        p_more = df_valid_pair["more_toxic_id"].map(lambda x: m[x] )
        return (p_less < p_more).mean()
    
    else:
        m = { a:b for a, b in zip(df_valid_text["text"].values, pred)  }
        
        df_valid_pair = df_valid_pair.copy()
        #df_valid_pair = df_valid_pair.query("count == 3").copy()
        
        df_valid_pair["less_toxic_score"] = df_valid_pair["less_toxic"].map(lambda x: m[x] )
        df_valid_pair["more_toxic_score"] = df_valid_pair["more_toxic"].map(lambda x: m[x] )
        
        d_a = df_valid_pair.query("less_toxic_score < more_toxic_score and avg_agreement == 1.0") 
        d_b = df_valid_pair.query("less_toxic_score < more_toxic_score and avg_agreement < 1.0") 
        d_c = df_valid_pair.query("less_toxic_score > more_toxic_score and avg_agreement < 1.0") 

        return (d_a["count"].sum() + d_b.shape[0]*2 + d_c.shape[0])/df_valid_pair["count"].sum()
    

def avg_predict (df_valid_text, cols):
    for k, col in enumerate(cols):
        p = rankdata(df_valid_text[col].values, method='ordinal')

        if k == 0:
            y = p
        else:    
            y = y + p  
    return y

def sklearn_predict (df_valid_text, model_path, model_name, cols, folds = 5):
    pred = np.zeros ((df_valid_text.shape[0]))
    X = df_valid_text [cols].values


    for fold in range(5):
        model = pickle.load(open(f"{model_path}/{model_name}_{fold}.pkl", 'rb'))

        pred += model.predict ( X )
        
    return pred

In [4]:
val_pair, val_text = create_validation_features ()
rud_pair, rud_text = create_rud_features ()

###  Mean Models

In [5]:
cols = ['original_toxicity', 'original_severe_toxicity', 
       'original_identity_attack', 
        'unbiased_toxicity', 'unbiased_severe_toxicity', 'unbiased_identity_attack', 'unbiased_insult', 
        'multilingual_toxicity', 'multilingual_sexual_explicit', 
        'original-small_toxicity', 
        'unbiased-small_severe_toxicity', 'unbiased-small_obscene', 
        'jc_tfidf_toxic']




y = avg_predict (rud_text, cols)
rud_score = validate (rud_pair,rud_text, y)

y = avg_predict (val_text, cols)
val_score = validate (val_pair,val_text, y)
print(f"Mean of {cols}\nVAL: {val_score:.5f}, RUD: {rud_score:.5f}")    

Mean of ['original_toxicity', 'original_severe_toxicity', 'original_identity_attack', 'unbiased_toxicity', 'unbiased_severe_toxicity', 'unbiased_identity_attack', 'unbiased_insult', 'multilingual_toxicity', 'multilingual_sexual_explicit', 'original-small_toxicity', 'unbiased-small_severe_toxicity', 'unbiased-small_obscene', 'jc_tfidf_toxic']
VAL: 0.70779, RUD: 0.81327


In [6]:
selected_cols = ['original_toxicity', 'original_severe_toxicity', 
       'original_identity_attack', 
        'unbiased_toxicity', 'unbiased_severe_toxicity', 'unbiased_identity_attack', 'unbiased_insult', 
        'multilingual_toxicity', 'multilingual_sexual_explicit', 
        'original-small_toxicity', 
        'unbiased-small_severe_toxicity', 'unbiased-small_obscene', 
        'jc_tfidf_toxic']

rud_gain = []
val_gain = []
for col in selected_cols:
    cols = [c for c in selected_cols if c != col]
    y = avg_predict (rud_text, cols)
    col_rud_score = validate (rud_pair,rud_text, y)
    
    col_rud_gain = rud_score - col_rud_score
    rud_gain.append ( col_rud_gain)
    
    y = avg_predict (val_text, cols)
    col_val_score = validate (val_pair,val_text, y)

    col_val_gain = val_score - col_val_score
    val_gain.append ( col_val_gain)

    
    print(f"{col} VAL: {col_val_gain:.5f}, RUD: {col_rud_gain:.5f}")    
    

original_toxicity VAL: 0.00017, RUD: 0.00102
original_severe_toxicity VAL: 0.00046, RUD: -0.00031
original_identity_attack VAL: -0.00080, RUD: 0.00440
unbiased_toxicity VAL: 0.00033, RUD: 0.00210
unbiased_severe_toxicity VAL: 0.00056, RUD: -0.00102
unbiased_identity_attack VAL: 0.00093, RUD: 0.00234
unbiased_insult VAL: 0.00060, RUD: 0.00391
multilingual_toxicity VAL: 0.00083, RUD: 0.00189
multilingual_sexual_explicit VAL: 0.00033, RUD: -0.00291
original-small_toxicity VAL: 0.00013, RUD: -0.00245
unbiased-small_severe_toxicity VAL: 0.00063, RUD: 0.00153
unbiased-small_obscene VAL: 0.00093, RUD: -0.00049
jc_tfidf_toxic VAL: -0.00033, RUD: -0.00564


In [7]:
df = pd.DataFrame ({
    "feature":selected_cols,
    "rud_importance" :rud_gain,
    "val_importance":val_gain,
}
)

df

,feature,rud_importance,val_importance
0,original_toxicity,0.001019,0.000166
1,original_severe_toxicity,-0.000310,0.000465
2,original_identity_attack,0.004399,-0.000797
3,unbiased_toxicity,0.002104,0.000332
4,unbiased_severe_toxicity,-0.001023,0.000565
5,unbiased_identity_attack,0.002336,0.000930
6,unbiased_insult,0.003913,0.000598
7,multilingual_toxicity,0.001891,0.000830
8,multilingual_sexual_explicit,-0.002912,0.000332
9,original-small_toxicity,-0.002455,0.000133
